<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/08_sentiment_analysis_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMを使ってみる
* 今日は、とりあえず、LLMを使ってみる。

## 今日の授業の目的
* いまどのくらい手軽にLLMを使えるようになっているかを、とりあえず体感する。
* 技術的な詳細は次回以降学んでいくことにして、とにかく使ってみる。

* **ランタイムのタイプをGPUに設定しておくこと。**

## 例題: LLMによる感情分析(sentiment analysis)
* 今日は、WRIMEというデータセットを使って、LLMに感情分析させてみる。
* 感情分析とは、テキストが表す感情を分析するタスク。
 * ポジティブな感情か、ネガティブな感情かの2値分類タスクとして解くことが多い。
 * 今日は、ニュートラルな感情も含めた3値分類問題として解くことにする。
* LLMとしてはELYZA-japanese-Llama-2-7b-instructを使う。
 * プロンプトを使ったテキスト生成によって感情分析の問題を解く。

## 準備
* Hugging Faceの各種ライブラリを使えば、簡単なコードを書くだけでLLMを使える。

### Transformersライブラリのインストール
* https://huggingface.co/docs/transformers/index

In [ ]:
!pip install transformers

### Datasetsライブラリのインストール
* https://huggingface.co/docs/datasets/index

In [ ]:
!pip install datasets

### Accelerateライブラリのインストール
* https://huggingface.co/docs/accelerate/index

In [ ]:
!pip install accelerate

### 量子化されたモデルを使うためのライブラリAutoGPTQのインストール
* https://huggingface.co/docs/optimum/llm_quantization/usage_guides/quantization
 * https://huggingface.co/blog/gptq-integration

In [ ]:
!pip install auto-gptq

**ここでランタイムを再起動する。**

### インポート

In [ ]:
import os
import torch
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.manual_seed(0)

## データセット
* Ver. 2 の方を使う。
 * WRIME: 主観と客観の感情分析データセット https://github.com/ids-cv/wrime
* Hugging Faceのdatasets hubに登録されているので、簡単に扱うことができる。

### WRIMEデータセットの取得
* training 30,000件、validation 2,500件、test 2,500件。
* 最初だけ、ダウンロードに時間がかかる。

In [ ]:
dataset = load_dataset("shunk031/wrime", "ver2")

In [ ]:
dataset

In [ ]:
dataset["validation"][0]

### 正解ラベルの確認

* 今回は、`avg_readers`の`sentiment`を正解ラベルとして使用する。

In [ ]:
from collections import Counter

labels = []
for example in dataset["validation"]:
  labels.append(example["avg_readers"]["sentiment"])
Counter(labels)

* 元のデータでは５値。
* 今回は、-2と-1をnegativeとして、2と1をpositiveとして、それぞれまとめることにする。
 * これで3値分類の問題になる。
 * このための前処理は後で行う。

## LLM


* 今回は、ELYZA-japanese-Llama-2-7b-instructを使う。
 * https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-instruct
* だが、Google Colab無料版では、この元のモデルは大きすぎて使えない・・・。
 * 20GB以上のメモリがあるGPUなら使える。
* そこで、量子化された下記のモデルを代わりに使う。
 * https://huggingface.co/TFMC/ELYZA-japanese-Llama-2-7b-instruct-GPTQ-4bit-64g

### ELYZA-japanese-Llama-2-7b-instruct-GPTQ-4bit-64gの取得
* モデルのダウンロードに少し時間がかかる。
* `AutoGPTQForCausalLM`クラスについては、以下を参照。
 * https://github.com/PanQiWei/AutoGPTQ/blob/main/auto_gptq/modeling/auto.py

* safetensorsについては、以下を参照。
 * https://huggingface.co/docs/diffusers/using-diffusers/using_safetensors

* `trust_remote_code`については、[ここ](https://huggingface.co/docs/transformers/model_doc/auto)に以下のような説明がある。

> Whether or not to allow for custom models defined on the Hub in their own modeling files. This option should only be set to True for repositories you trust and in which you have read the code, as it will execute code present on the Hub on your local machine.



In [ ]:
from auto_gptq import AutoGPTQForCausalLM

model_name = "TFMC/ELYZA-japanese-Llama-2-7b-instruct-GPTQ-4bit-64g"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoGPTQForCausalLM.from_quantized(
    model_name,
    use_safetensors=True,
    inject_fused_attention=False,
    device="cuda:0",
    #trust_remote_code=True,
    )
model.eval()

## プロンプト
* LLMがうまく感情分析をしてくれそうなプロンプトを考える。
 * 下はあくまで一つの例。

### プロンプト作成用のヘルパ関数

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]答え："
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントで、人の感情を察するのが得意です。"

def make_prompt(example):
  sentence = example['sentence']
  text = "「" + sentence + "」と言っている人の気持ちは、どのように説明できますか。"
  text += "また、その気持ちを１単語で言うと、次のどちらですか：「嬉しい」、「悲しい」。"
  text += "どちらでもなければ、「どちらでもない」と答えてください。"
  prompt = "{b_inst} {system}{prompt} {e_inst} ".format(
      b_inst=B_INST,
      system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
      prompt=text,
      e_inst=E_INST,
      )
  example['sentence'] = prompt
  return example

* 元のテキストをプロンプトに一括変換する。

In [ ]:
validation_set = dataset["validation"].map(make_prompt)

In [ ]:
print(validation_set[0]["sentence"])

* プロンプトをあらかじめトークン化しておく。

In [ ]:
validation_set = validation_set.map(lambda samples: tokenizer(samples['sentence']), batched=True)

* `input_ids`というフィールドにトークン化の結果が格納されている。

In [ ]:
print(validation_set[0]["input_ids"])

In [ ]:
print(validation_set[0]["sentence"])

In [ ]:
tokenizer.convert_ids_to_tokens([1])

## 感情分析

### 5値を3値に変換するヘルパ関数

In [ ]:
def get_sentiment(example):
  sentiment_dic = {-2:'悲しい', -1:'悲しい', 0:'どちらでもない', 1:'嬉しい', 2:'嬉しい'}
  sentiment = example['avg_readers']['sentiment']
  return sentiment_dic[sentiment]

### 感情分析の実行

In [ ]:
for i in range(10):
  print(f'[{i+1}]' + '-'*80)
  instance = validation_set[i]
  prompt = instance["sentence"]
  with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    output_ids = model.generate(
        input_ids=token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
  print(f"{prompt}\nprediction:\t{output}")
  print(f"ground truth:\t「{get_sentiment(instance)}」")
  print('-'*80)

# 本日の課題
* もっとうまくLLMに感情分析をさせるプロンプトを考えてみよう。